In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [3]:
if not os.path.exists('./mlp_img_conv'):
    os.mkdir('./mlp_img_conv')

In [6]:
def to_img(x):
    x = x.view(x.size(0),1,28,28)
    #print (x)
    return x

In [7]:
num_epochs = 100
batch_size = 128
learning_rate = 0.001

In [8]:
def plot_sample_img(img, name):
    img=img.view(1,28,28)
    save_img(img, './sample_{}.png'.format(name))

In [9]:
def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor/max_tensor
    tensor = tensor*(max_value - min_value) + min_value
    return torch.round(tensor)

In [10]:
img_transform =transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [11]:
dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle= True)

In [10]:
'''class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28,256),
            nn.ReLU(True),
            nn.Linear(256,64),
            nn.ReLU(True))
        
        self.decoder = nn.Sequential(
            nn.Linear(64,256),
            nn.ReLU(True),
            nn.Linear(256,28*28),
            nn.Sigmoid())
        # Define forward
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
'''
#Convolution autoencoder
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
model = autoencoder().cuda()
model

autoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(8, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace)
    (2): ConvTranspose2d(16, 8, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (3): ReLU(inplace)
    (4): ConvTranspose2d(8, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (5): Tanh()
  )
)

In [12]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=0.0001)

In [13]:
# Training

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        #img = img.view(img.size(0),-1)
        img = Variable(img).cuda()
        
        # Forward
        output = model(img)
        loss = criterion(output, img)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # log
    print('epoch [{}/{}], loss:{:.4f}'
        .format(epoch + 1, num_epochs, loss.item()))
    if epoch%10==0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        save_image(x,'./mlp_img_conv/x_{}.png'.format(epoch))
        save_image(x,'./mlp_img_conv/x_hat{}.png'.format(epoch))
            
torch.save(model.state_dict(),'./sim_denoising_autoencoder.pth')
        

epoch [1/100], loss:0.2093
epoch [2/100], loss:0.1584
epoch [3/100], loss:0.1435
epoch [4/100], loss:0.1385
epoch [5/100], loss:0.1288
epoch [6/100], loss:0.1190
epoch [7/100], loss:0.1216
epoch [8/100], loss:0.1159
epoch [9/100], loss:0.1176
epoch [10/100], loss:0.1186
epoch [11/100], loss:0.1114
epoch [12/100], loss:0.1106
epoch [13/100], loss:0.1076
epoch [14/100], loss:0.1056
epoch [15/100], loss:0.1189
epoch [16/100], loss:0.0982
epoch [17/100], loss:0.1087
epoch [18/100], loss:0.1130
epoch [19/100], loss:0.1047
epoch [20/100], loss:0.0969
epoch [21/100], loss:0.0997
epoch [22/100], loss:0.1046
epoch [23/100], loss:0.1052
epoch [24/100], loss:0.1039
epoch [25/100], loss:0.1027
epoch [26/100], loss:0.1047
epoch [27/100], loss:0.1018
epoch [28/100], loss:0.1041
epoch [29/100], loss:0.1039
epoch [30/100], loss:0.1069
epoch [31/100], loss:0.1001
epoch [32/100], loss:0.0944
epoch [33/100], loss:0.0951
epoch [34/100], loss:0.0873
epoch [35/100], loss:0.1026
epoch [36/100], loss:0.0993
e

In [14]:
os.getcwd()

'D:\\1st year\\Python codes\\CIFR Analysis\\self contained analysis'